In [1]:
import requests
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

API_KEY = 'c1lq1mzoyp1z0nx6bsqi9lrk'
request_data = {'api_key':'c1lq1mzoyp1z0nx6bsqi9lrk'}

listing_id = 570279570
request = str('https://openapi.etsy.com/v2/listings/'+str(listing_id))
r = requests.get(request, request_data).json()

In [2]:
# Load models and data structures
views_est = joblib.load('tfidf_views_GradientBoostingRegressor.pkl') 
price_est = joblib.load('tfidf_price_GradientBoostingRegressor.pkl') 
lda = joblib.load('tf_lda.pkl') 
nmf = joblib.load('tfidf_nmf.pkl')
listings = joblib.load('listings.pkl')
features = joblib.load('all_features.pickle')
tf_features = joblib.load('tf_features.pkl')
tfidf_features = joblib.load('tfidf_features.pkl')
tf_vectorizer = joblib.load('tf_vectorizer.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

tf_feature_names = tf_vectorizer.get_feature_names()
prices = np.nan_to_num(listings['price'].tolist())
views = np.nan_to_num(listings['views'].tolist())
favorers = np.nan_to_num(listings['num_favorers'].tolist())
test_size = 0.1

tfidf_price_train, tfidf_price_test, y_price_train, y_price_test = train_test_split(
    tfidf_features, prices, test_size = test_size)

tfidf_views_train, tfidf_views_test, y_views_train, y_views_test = train_test_split(
    tfidf_features, views, test_size = test_size)

n_features = 1000 # Number of features
n_samples = 36570 # Number of samples
n_components = 100 # Number of components
n_top_words = 10  # Number of top words

/home/mc/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Define functions

In [3]:
# Print the top words for each topic in a model (NMF or LDA in this case)
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [4]:
# Return a list of top features for a given topic group in a model
def model_topic_features(model, feature_names, n_top_words, topic_group):
    for topic_idx, topic in enumerate(model.components_):
        if (topic_idx == topic_group):
            result = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return(result)

In [5]:
def find_similar_features(model, features):
    topic_list = model.transform(tf_vectorizer.transform([features]))[0].argmax()
    topic_features = model_topic_features(model, tf_feature_names, 50, topic_list)
    new_features = [item for item in topic_features.split() if item not in features.split()]
    return(new_features)

In [6]:
# Get features (tags + materials) for a given Etsy ID
def get_features_from_listing_id (listing_id):
    request = str('https://openapi.etsy.com/v2/listings/'+str(listing_id))
    r = requests.get(request, request_data).json()
    tags = " ".join(r.get('results')[0].get('tags')).lower()
    materials = " ".join(r.get('results')[0].get('materials')).lower()
    features = tags+" "+materials
    return features

In [95]:
# Get price, views, and number of favorers for a given Etsy ID
def get_listing_info(listing_id):
    request = str('https://openapi.etsy.com/v2/listings/'+str(listing_id))
    r = requests.get(request, request_data).json()
    original_creation_date = r.get('results')[0].get('original_creation_tsz')
    original_creation_date = datetime.strptime(datetime.utcfromtimestamp(original_creation_date).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S")
    ending_date = r.get('results')[0].get('ending_tsz')
    ending_date = datetime.strptime(datetime.utcfromtimestamp(ending_date).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S")
    difference = (ending_date - original_creation_date).days
    price = r.get('results')[0].get('price')
    views = r.get('results')[0].get('views')
    views = views / difference
    num_favorers = r.get('results')[0].get('num_favorers')
    num_favorers = num_favorers / difference
    return([price, views, num_favorers, difference])

In [96]:
def find_similar_features_from_etsy_id(etsy_id, model):
    original_features = get_features_from_listing_id(etsy_id)
    features = " ".join(tfidf_vectorizer.inverse_transform(tfidf_vectorizer.transform([original_features]))[0])
    similar_features = " ".join(find_similar_features(model, features)[:10])
    return [features, similar_features]

In [101]:
def return_model_info(etsy_id):
    feature_info = find_similar_features_from_etsy_id(etsy_id, lda)
    features = feature_info[0]
    similar_features = feature_info[1]
    listing_info = get_listing_info(etsy_id)
    original_price = listing_info[0]
    original_price_prediction = price_est.predict(tf_vectorizer.transform([features]))[0]
    original_views = listing_info[1]
    original_views_prediction = views_est.predict(tf_vectorizer.transform([features]))[0]
    num_favorers = listing_info[2]
    print(features)
    print(similar_features)
    print(lda.score(similar_features, features))
    return[features, similar_features, original_price, original_price_prediction, original_views, original_views_prediction, num_favorers]

In [55]:
request = str('https://openapi.etsy.com/v2/listings/'+str(listing_id))
r = requests.get(request, request_data).json()

In [67]:
r.get('results')[0].get('num_favorers')

546

In [93]:
from datetime import datetime

In [106]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=100, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [45]:
print(" ".join(tfidf_vectorizer.inverse_transform(tfidf_vectorizer.transform([original_features]))[0]))

yarn twins twin sock prop photography photo newborn monkey hat girl earflaps buttons boy baby babies


In [35]:
test = nmf.transform(tf_vectorizer.transform([features[0]]))

In [79]:
get_listing_info(280073754)

['15.00', 114104, 12315]

In [81]:
return_model_info(280073754)

['women personalized monogrammed monogram hats hat gift embroidered cap baseball',
 'sun cotton felt fabric beach flowers cowboy thread ribbon upcycled',
 '15.00',
 22.747132598586468,
 114104,
 1514.9611808047089,
 12315]

In [75]:
similar_features = find_similar_features_from_etsy_id(280073754, lda)
print("original_features: ", similar_features[0])
print("new_features: ", similar_features[1])
print("predicted price: ",price_est.predict(tf_vectorizer.transform([similar_features[0]])))

original_features:  women personalized monogrammed monogram hats hat gift embroidered cap baseball
new_features:  sun cotton felt fabric beach flowers cowboy thread ribbon upcycled
predicted price:  [22.7471326]


In [48]:
similar_features = find_similar_features_from_etsy_id(280073754, nmf)
print("original_features:", similar_features[0])
print("\nnew_features:", similar_features[1])

original_features: women personalized monogrammed monogram hats hat gift embroidered cap baseball

new_features: stocking chemo skull pima skullcap wolle spirit infant watch rasta


In [49]:
similar_features = find_similar_features_from_etsy_id(457283538, lda)
print("original_features:", similar_features[0])
print("\nnew_features:", similar_features[1])

original_features: yarn winter warmer toque prop pom photo knitted knit kids hat girls ear crocheted boys beanies beanie baby

new_features: newborn photography boy girl props infant acrylic shower gift button


In [50]:
similar_features = find_similar_features_from_etsy_id(457283538, nmf)
print("original_features:", similar_features[0])
print("\nnew_features:", similar_features[1])

original_features: yarn winter warmer toque prop pom photo knitted knit kids hat girls ear crocheted boys beanies beanie baby

new_features: flap designer flaps snowboard clothing earflaps warmers adults rhinestones apparel


In [51]:
similar_features = find_similar_features_from_etsy_id(561078472, lda)
print("original_features:", similar_features[0])
print("\nnew_features:", similar_features[1])

original_features: yellow slouch satin mustard hat cotton cap beanie

new_features: green yarn soft acrylic washable handmade bright color knit olive


In [52]:
similar_features = find_similar_features_from_etsy_id(561078472, nmf)
print("original_features:", similar_features[0])
print("\nnew_features:", similar_features[1])

original_features: yellow slouch satin mustard hat cotton cap beanie

new_features: minion orange despicable gold bee neon bright stripes chick ribbon


In [ ]:
from datetime import datetime
start = listings['original_creation_tsz'].tolist()
end = listings['ending_tsz'].tolist()
d_start = [datetime.strptime(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S") for i, x in enumerate(start)]
d_end = [datetime.strptime(datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'), "%Y-%m-%d %H:%M:%S") for i, x in enumerate(end)]
difference = [(d_end[i] - d_start[i]).days for i in range(len(d_start))]
listings = listings.assign(days=pd.Series(difference))
views = listings['views'].tolist()
num_favorers = listings['num_favorers'].tolist()
days = listings['days'].tolist()
views_days = [views[i] / days[i] for i in range(len(views))]
num_favorers_days = [num_favorers[i] / days[i] for i in range(len(num_favorers))]
listings = listings.assign(views=pd.Series(views_days))
listings = listings.assign(num_favorers=pd.Series(num_favorers_days))
listings.to_pickle("./listings.pkl")